In [1]:
import numpy as np
import pandas as pd
import data_collection as dc
import psycopg2
import datetime


In [41]:
## code to open sql files. Dont need it now

# file_obj = open('../src/sql/05_pull_extr_rpsale_2019.sql')

# file_contents = file_obj.read()
# file_obj.close()

# file_contents
# df_test = pd.read_sql(file_contents, conn)

In [2]:
dbname = "king_house"

In [3]:
conn = psycopg2.connect(dbname = dbname)

In [4]:
df_sale = pd.read_sql("""
SELECT *

FROM extr_rpsale
""", conn)

In [5]:
df_rdb = pd.read_sql("""
SELECT *

FROM extr_resbldg
""", conn)

In [6]:
df_parcel = pd.read_sql("""
SELECT *

FROM extr_parcel

""", conn)

In [7]:
df_lookup = pd.read_sql("""
SELECT *


FROM look_up

""", conn)

In [100]:
df_lookup = df_lookup[df_lookup['LUType'] != 'LUType']

In [101]:
# Selcted variables from the residenstial building dataset:

df_rdb = df_rdb[df_rdb['Major'] != 'Major']
df_rdb['HID'] = df_rdb['Major'] + df_rdb['Minor']

columns = ['Major', 'Minor', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
            'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving',
            'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade','SqFtGarageBasement', 
            'SqFtGarageAttached', 'DaylightBasement','SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck',
            'Bedrooms','BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'HID']
df_rdb_set = df_rdb.loc[:, columns]

In [102]:
# Selected variables from the parcel dataset:
df_parcel = df_parcel[df_parcel['Major'] != 'Major']
df_parcel['HID'] = df_parcel['Major'] + df_parcel['Minor']

columns = ['Major', 'Minor', 'SqFtLot','PugetSound', 'LakeWashington','LakeSammamish', 
           'SmallLakeRiverCreek', 'TrafficNoise', 'AirportNoise', 'PowerLines', 'HID'] 
df_parcel_set = df_parcel.loc[:, columns]
df_parcel_set.head()

,Major,Minor,SqFtLot,PugetSound,LakeWashington,LakeSammamish,SmallLakeRiverCreek,TrafficNoise,AirportNoise,PowerLines,HID
1,889250,80,10848,0,0,0,0,0,0,N,88925080
2,736360,275,9900,0,3,0,0,0,0,N,736360275
3,600350,635,610,0,0,0,0,0,0,N,600350635
4,635260,760,19007,0,0,0,0,0,0,N,635260760
5,333250,15,6600,0,0,0,0,0,0,N,33325015


In [103]:
# Selseted variables from the property sale dataset: 

df_sale = df_sale[df_sale['Major'] != 'Major']
df_sale['Year'] = df_sale['DocumentDate'].str[6:]
df_sale_2019 = df_sale[df_sale['Year'] == '2019']
df_sale['HID'] = df_sale['Major'] + df_sale['Minor']

columns = ['Major', 'Minor', 'DocumentDate', 'SalePrice', 'PropertyType', 'HID']
df_sale_2019_set = df_sale_2019.loc[:, columns]

In [114]:
# Join 3 datasets together

df_combine = df_sale_2019_set.merge(df_parcel_set, left_on='HID', right_on='HID').merge(df_rdb_set, left_on='HID', right_on='HID')

In [113]:
df_combine.columns

Index(['Major_x', 'Minor_x', 'DocumentDate', 'SalePrice', 'PropertyType',
       'HID', 'Major_y', 'Minor_y', 'SqFtLot', 'PugetSound', 'LakeWashington',
       'LakeSammamish', 'SmallLakeRiverCreek', 'TrafficNoise', 'AirportNoise',
       'PowerLines', 'Major', 'Minor', 'SqFt1stFloor', 'SqFtHalfFloor',
       'SqFt2ndFloor', 'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf',
       'SqFtTotLiving', 'SqFtTotBasement', 'SqFtFinBasement',
       'FinBasementGrade', 'SqFtGarageBasement', 'SqFtGarageAttached',
       'DaylightBasement', 'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck',
       'Bedrooms', 'BathHalfCount', 'Bath3qtrCount', 'BathFullCount'],
      dtype='object')